# Micro-Ion Plus Vaccum Gauge: Test Scripts

Updated: 2025 Aug 7 [rwp/osu]

This notebook contains simple python functions that you can use to interact with the ion gauge.

## Importing Libraries and Defining Constants

 * `GAUGE_IP` is the IP address of the Comtrol Device Master connected via RS485 to the ion gauge
 * `GAUGE_PORT` is the port on the Comtrol, usually 8018

For MODS, the Comtrol IP addresses are as follows.  All are port **8018**
 * MODS1R: **192.168.139.103**
 * MODS1B: **192.168.139.113**
 * MODS2R: **192.168.139.203**
 * MODS2B: **192.168.139.213**

All MODS ion gauges are now confirmed (and set) to RS485 channel **5**.

In [86]:
import socket

#The address:port of an ion-guage.
GAUGE_IP = "192.168.139.103"
GAUGE_PORT = 8018

#The maximum length of a string returned by the ion gauge.
MESSAGE_SIZE = 16

#The maximum amount of time to wait in seconds before aborting socket operations.
TIMEOUT = 5

## Defining Two Useful Functions

**initIonSocket(address, port, timeout)**
- address: The IP address of the ion-gauge.
- port: The port on which the ion-gauge is connected.
- timeout (optional): The maximum amount of time to wait in seconds before the socket throws an exception.
- *return*: A TCP socket which has been connected to an ion-gauge.
- *An exception will be thrown if a connection can not be made, or if the ion-gauge is unresponsive.*
    
    
**sendIonCommand(ionSocket, command)**
- ionSocket: A TCP socket which has been connected to an ion-gauge.
- command: A string containing the command which will be sent to the ion-gauge.
- *return*: A string with the response message from the ion gauge.
- *An exception will be thrown if the command can not be sent, or if a response is never recieved.*

In [87]:
def sendIonCommand(ionSocket, command):
    #Send and recieve data.
    try:
        ionSocket.send((command).encode())
        data = ionSocket.recv(MESSAGE_SIZE).decode()
    except socket.error:
        raise Exception("Could not send command to ion-guage.")

    #Return the collected data.
    return data

def initIonSocket(address, port, timeout="None"):
    #Creating the socket
    ionSocket = socket.socket()

    #Setting a timeout in seconds
    if(timeout != "None"):
        ionSocket.settimeout(timeout)
    
    #Open the TCP connection.
    try:
        ionSocket.connect((address, port))
    except socket.error:
        ionSocket.close()
        raise Exception("Could not connect to ion-gauge.")
    
    #Attempting an initial snd/rcv to test connection.
    try:
        sendIonCommand(ionSocket, "#05VER\r")
    except:
        ionSocket.close()
        raise Exception("Connection opened, but communication was not possible. Is the gauge powered on?")
    
    #Return the connected socket.
    return ionSocket

## Usage Instructions

To interact with the ion-gauge you should: 
1. Initialize a socket
2. Send commands / Receive responses
3. Close the socket

When sending commands, ensure that you have included a carriage return `\r` at the end of the command.
<br/><br/>

**Commands that do not affect module operation:**
| Command    | Purpose            | Example Response | Description                                                                  | 
| ---------- | ------------------ | ---------------- | ---------------------------------------------------------------------------- |
| `#05IGS\r` | On/Off Status      | `*05 1 IG ON`    | "1 IG ON" if the gauge is on. "0 IG OFF" if the gauge is off.                |
| `#05KBS\r` | Keyboard Status    | `*05 1 KB OFF`   | "1 KB ON" if the buttons are locked. "1 KB OFF" if the buttons are unlocked. |
| `#05RD\r`  | Read Pressure      | `*05 9.97E+02`   | Returns measured pressure in the units set on the module (usually torr)      |
| `#05RE\r`  | Read Emission      | `*05 4.0MA EM`   | Returns an emission string. See table 3-15 on page 48.                       |
| `#05RF\r`  | Read Filament      | `*05 FIL SF1`    | "FIL SF1", "FIL SF2", or "FILBOTH" to indicate which filament is operating.  |
| `#05RS\r`  | Read Module Status | `*05 00 ST OK`   | Returns a module status string. See table 3-16 on page 49.                   |
| `#05VER\r` | Get Module Version | `*05 14851-06`   | Returns the part number, a dash, and then the firmware version.              |

*Here it is assumed that the RS485 address is set to 5 - that is why all commands and responses are prefixed with 05.*
<br/><br/>

**Commands that *do* affect module operation:**

For commands that do affect module operation, see page 50.

## Example Script

In [91]:
try:
    #Step 1 - Initalizing the socket.
    ionSocket = initIonSocket(GAUGE_IP, GAUGE_PORT, TIMEOUT)

    #Step 2 - Send commands and recieve responses.
    print(sendIonCommand(ionSocket, "#05RD\r"))   # Will print the current gauge pressure reading.
    print(sendIonCommand(ionSocket, "#05RS\r"))   # Will print the current status.
    print(sendIonCommand(ionSocket, "#05VER\r"))  # Will print the current version.
    print(sendIonCommand(ionSocket, "#05IGS\r"))  # will print gauge power status on/off
    print(sendIonCommand(ionSocket, "#05RF\r"))  # will print filament status

    #Step 3 - Closing the socket.
    ionSocket.close()
except Exception as e:
    print("[Error]: " + str(e))

*05 1.66E-01
*05 00 ST OK
*05 14851-06
*05 0 IG OFF
*05 FIL  SF1
